In [0]:
from collections import defaultdict
from matplotlib import pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
import torch

def one_hot_to_text(one_hot, languageModel, filter_unk=False):
  return [languageModel.vocab.itos[idx] for idx in one_hot if not (filter_unk and idx == 0)]

def experiment(model, iterator):
    counts = defaultdict(float)
    scores = defaultdict(float)

    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            if i != 0: break
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 1)
            sentence_size, batch_size, vocab_size = output.shape
            for batch_idx in range(batch_size):
                probs = F.softmax(output[:,batch_idx,:], 1)
                print(probs.shape)
                _, sentence_by_idx = probs.max(axis=1)
                print(sentence_by_idx)
                    
                score = sentence_bleu([one_hot_to_text(trg[:,batch_idx], targetLanguage)], one_hot_to_text(sentence_by_idx, targetLanguage))
                counts[len(src.view(-1))] += 1
                scores[len(src.view(-1))] += score

    return {length: scores[length] / counts[length] for length in scores}

def plot_scores(model, iterator):
    results = experiment(model, iterator)
    ks = [k for k in results]
    plt.plot(ks, [results[k] for k in ks])

In [0]:
def beam_search(beam_width, max_length, model, batch, sourceLanguage, targetLanguage):
    batch_size = batch.trg.shape[1]
    outputs = torch.zeros(batch_size, max_length, dtype=torch.long)
    encoder_output, encoder_hidden = model.encoder(batch.src)

    for i in range(batch_size):
        batch_encoder_output = encoder_output[:, i, :].unsqueeze(1)
        beam = [(0.0, torch.LongTensor([sourceLanguage.vocab.stoi[sourceLanguage.init_token]]), encoder_hidden[i,:].unsqueeze(0))]
        
        for t in range(1, max_length):
            nxt = []
            all_terminated = 0 # to be used to detect if all viable sentences have hit eos

            for node in beam:
                score, seq, hidden = node

                # To check if this viable sentence has already hit an eos
                if seq[-1] == targetLanguage.vocab.stoi[targetLanguage.eos_token]:
                    all_terminated += 1
                    nxt += [node]
                    continue

                decoder_output, decoder_hidden = model.decoder(torch.LongTensor([seq[-1]]).to(device), hidden, batch_encoder_output)
                probs = F.softmax(decoder_output.squeeze())
                log_probs = torch.log(probs)
                top_scores, top_idxs = log_probs.topk(beam_width)
                nxt += [(score + top_scores[b], torch.cat([seq, torch.LongTensor([top_idxs[b]])]), decoder_hidden) for b in range(beam_width)]

            if all_terminated == beam_width: break # to check for termination
            beam = sorted(nxt, reverse=True, key=lambda x: x[0])[:beam_width]

        _, output_vec, _ = max(beam, key=lambda x: x[0])
        sentence_length = output_vec.shape[0]
        outputs[i,:sentence_length] = output_vec
    return outputs